#Pipeline de Análisis de Sentimientos. 

Importaciones y carga. 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

Carga del archivo csv. 

In [3]:
df = pd.read_csv('../data/raw/comentarios.csv')

Imputación de nulos

In [4]:
print(df.isnull().sum())

id            0
usuario       0
comentario    0
fecha         0
fuente        0
dtype: int64


En base al output obtenido podemos observar que existen valores nulos dentro del dataset. Ellos son:
- 4 valores nulos en la columna usuario. 
- 20 valores nulos en la columna comentario. 
- 11 valores nulos en la columna fuente. 

Abordaje para la resolución de los valores faltantes. 

- Valores faltantes en la columna 'usuario'. 
Sobre la misma de plantea crear una categoría que permita mantener la trazabilidad, a través de un nombre 'Anonimo_NA'. 
De esta manera evitamos sesgar los datos  generamos una categoría que nos permitirá mantener la trazabilidad de los datos y la robustez del análisis. 

- Valores faltantes para la columna 'comentario'. 
Sobre la misma se plantea crear una categoría que permita mantener la trazabilidad, a través de una cadena de texto 'No aplica'. 
De esta manera evitamos sesgar los datos  generamos una categoría que nos permitirá mantener la trazabilidad de los datos y la robustez del análisis.

- Valores faltantes para la columna 'fuente'. 
El abordaje es el mismo que en las situaciones anteriormente descriptas. Nombre de la categoría 'Sin fuente'. 


In [5]:
#Rellenado de valores nulos en la columna 'usuario' con una cadena vacía
df['usuario'] = df['usuario'].fillna('Anonimo_NA')
#Rellenado de valores nulos en la columna 'comentario' con una cadena vacía
df['comentario'] = df['comentario'].fillna('No aplica')
#Rellenado de valores nulos en la columna 'fuente' con una cadena vacía
df['fuente'] = df['fuente'].fillna('Sin fuente')


In [6]:
#Verificamos la existencia de valores nulos luego del rellenado. 
df.isnull().sum()

id            0
usuario       0
comentario    0
fecha         0
fuente        0
dtype: int64

Como se muestra en el output obtenido, la imputación de nulos resultó favorable, eliminando los valores vacios. 

Imputación de strings. 

Limpieza del texto, se procede a: 
- Eliminar espacios vacios al principio y al final de las columnas del tipo str.
- Convertir a minúsculas. 
- Eliminar caracteres especiales innecesarios. 

In [7]:
#Eliminación  de espacios en blanco. 
df['usuario'] = df['usuario'].str.strip()
df['comentario'] = df['comentario'].str.strip()
df['fuente'] = df['fuente'].str.strip()

In [8]:
#Convertir a minusculas. 
df['usuario'] = df['usuario'].str.lower()
df['comentario'] = df['comentario'].str.lower()
df['fuente'] = df['fuente'].str.lower()

In [9]:
#Detectar caracteres no alfabéticos en la columna 'comentario'
def detectar_no_alfabeticos(df, columna):
    patron =  r'[!?\@\#\$\%\^\&\*\|]'
    filtro = df[columna].str.contains(patron, regex = True)
    return df.loc[filtro, columna]

In [10]:
resultado = detectar_no_alfabeticos(df, 'comentario')
print(resultado)

Series([], Name: comentario, dtype: object)


La detección de caracteres no alfabeticos, verifica la presencia de un caracter incorrecto en la fila 32 '32    me encantó el servicio, tx|ntó el servicio, to...'. 
Al ser una única fila, se procede a corregir el dato de manera manual, quedando el comentario estructurado de la siguiente manera 'Me encantó el servicio, todo muy rápido'. 



Imputación de fechas. 

In [11]:
print(df.fecha.head(20))

0     2025-01-01
1     2025-01-02
2     2025-01-02
3     2025-01-04
4     2025-01-04
5     2025-01-06
6     2025-01-07
7     2025-01-08
8     2025-01-09
9     2025-01-10
10    2025-01-11
11    2025-01-12
12    2025-01-13
13    2025-01-14
14    2025-01-14
15    2025-01-16
16    2025-01-17
17    2025-01-18
18    2025-01-19
19    2025-01-20
Name: fecha, dtype: object


Como se muestra en el output anterior, vemos que existen varios problemas con los datos almacenados en la columna 'fecha'. 
- Fechas imposibles: 2025-13-40
- No estandarización del formato: 2025-01-02 y 03-01-2025.




In [12]:
#Estandarizar formato de fecha.
df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')

In [13]:
#verificar las conversiones erroneas como NaT
fechas_erroneas = df[df['fecha'].isna()]
print(fechas_erroneas)

Empty DataFrame
Columns: [id, usuario, comentario, fecha, fuente]
Index: []


In [14]:
#Aplicamos el método Forward Fill para rellenar los valores NaT
df['fecha'] = df['fecha'].fillna(method='ffill')

C:\Users\Juan Pablo\AppData\Local\Temp\ipykernel_9664\3166165498.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['fecha'] = df['fecha'].fillna(method='ffill')


In [15]:
#verificar las conversiones de fecha luego del método Forward Fill
fechas_erroneas = df[df['fecha'].isna()]
print(fechas_erroneas)

Empty DataFrame
Columns: [id, usuario, comentario, fecha, fuente]
Index: []


In [20]:
df.to_csv('../data/processed/comentarios_limpio.csv', index=False)